In [ ]:
from rdflib import Graph
import os
import pandas as pd

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def extract_favel_triples(path):
        g = Graph()
        g.parse(path)
        for s, p, o in g.triples((None,  None, None)):
            return str(s), str(p), str(o)

In [ ]:
import os
def getFavel(path):
        triples = pd.DataFrame(data=[], columns=['subject','predicate','object','truth'])
        paths = [os.path.join(path,'Turtle/Test/Correct/'), 
                 os.path.join(path,'Turtle/Test/Wrong/'), 
                 os.path.join(path,'Turtle/Train/Correct/'), 
                 os.path.join(path,'Turtle/Train/Wrong/')]
        for p in paths:
            for root, dirs, files in os.walk(p):
                for name in files:
                    triple = extract_favel_triples(root+'/'+name)
                    if p.find("Correct") != -1:
                        triple=triple+(str(1),)
                    else:
                        triple=triple+(str(0),)
                    triples = triples.append({'subject':triple[0], 'predicate':triple[1], 'object':triple[2], 'truth':triple[3]}, ignore_index=True)
        return triples

In [ ]:
def getInput():
        result = []
        df = getFavel("/Users/palaniappanmuthuraman/Documents/favel_private/Datasets/FinalDataset")
        result = parseTriples(df)
        return result
            
def parseTriples(df):
        result = []
        for i, (s,p,o,t) in df.iterrows():
            result.append((s,p,o,t))
        return result

In [ ]:
datas = getInput()
favelDataframe = pd.DataFrame(datas,columns=['subject','predicate','object','truth_value'])
favelDataframe.to_csv('favel_dataset.csv',index=False)

In [ ]:
def extract_bpdp_triples(file):
        subject = ""
        object_g = ""
        g = Graph()
        g.parse(file)
        if file.find("birth") != -1:
            predicate1 = "http://dbpedia.org/ontology/birth"
            predicate2 = "http://dbpedia.org/ontology/birthPlace"
        else:
            predicate1 = "http://dbpedia.org/ontology/death"
            predicate2 = "http://dbpedia.org/ontology/deathPlace"
        for s, p, o in g.triples((None,  None, None)):
            if str(p) == predicate1:
                subject=str(s)
            if str(p) == predicate2:
                object_g=str(o)
        return subject, predicate2, object_g

In [ ]:
def getBPDP(path):
        
        triples = pd.DataFrame(data=[], columns=['subject','predicate','object','truth'])
        paths = [os.path.join(path,'Test/True/'), 
                 os.path.join(path,'Test/False/'), 
                 os.path.join(path,'Train/True/'), 
                 os.path.join(path,'Train/False/')]

        for p in paths:
            for root, dirs, files in os.walk(p):
                for name in files:
                    
                    triple = extract_bpdp_triples(p+name)
                    if p.find("True") != -1:
                        triple=triple+(str(1),)
                    else:
                        triple=triple+(str(0),)
                    triples = triples.append({'subject':triple[0], 'predicate':triple[1], 'object':triple[2], 'truth_value':triple[3]}, ignore_index=True)
        return triples

In [ ]:
bpdpdata = getBPDP("/Users/palaniappanmuthuraman/Documents/favel_private/Datasets/BPDP_Dataset")
bpdpdataFrame = pd.DataFrame(bpdpdata,columns=['subject','predicate','object','truth_value'])
bpdpdataFrame.to_csv('bpdp_dataset.csv',index=False)

In [ ]:
def extract_ids(graph):
        ids = []
        for s,_,_ in graph:
            ids.append(s)
        return list(set(ids))

In [ ]:
def getFactbench(path):
        graph = Graph()
        
        for root, dirs, files in os.walk(path):
            for name in files:
                graph.parse(path+name, format="nt")
                
        ids = extract_ids(graph)
        
        triples = pd.DataFrame(data=[], columns=['subject','predicate','object','truth'])
        for id in ids:
            for _,p,o in graph.triples((id, None, None)):
                if str(p) == "http://swc2017.aksw.org/hasTruthValue":
                    if(str(o)=='1.0'):
                        truth = 1
                    else:
                        truth = 0
                if str(p).find("object") != -1:
                    object_elt=str(o)
                if str(p).find("predicate") != -1:
                    predicate=str(o)
                if str(p).find("subject") != -1:
                    subject=str(o)
            triples = triples.append({'subject':subject, 'predicate':predicate, 'object':object_elt, 'truth_value':truth}, ignore_index=True)

        return triples

In [ ]:
factBenchData = getFactbench("/Users/palaniappanmuthuraman/Documents/favel_private/Datasets/factbench/")
factBenchdataFrame = pd.DataFrame(factBenchData,columns=['subject','predicate','object','truth_value'])
factBenchdataFrame.to_csv('factbench_dataset.csv',index=False)